<a href="https://colab.research.google.com/github/gt-cse-6040/comparing-base-objects/blob/main/comparing_base_objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing Basic Python Objects

You will often find yourself in a situation where you need to determine if two Python objects are "the same", or more specifically that a Python variable that you generated is the same as a known, available result. This is applicable both for academic assignments with automated grading or in real-world use cases where you may have to refactor code to use new technology but require the same results as the legacy code.

We can check whether two basic Python objects are equivalent by verifying that they are the same `type` and verifying that they have the same `value`. We will define some objects and demonstrate comparing them.

In [ ]:
int_a = 2004
int_b = 2012
int_c = 2004

float_a = 2004.0
float_b = 2012.0
float_aa = 2004 + 10**(-12)

bool_a = True
bool_b = True
bool_c = False

str_a = "2004"
str_b = "2012"
str_c = "2004"


Now we can put them in an object to make it easier to compare them later.

In [ ]:
test_vars = {'int_a':int_a, 'int_b':int_b, 'int_c':int_c, 
             'float_a':float_a, 'float_b':float_b, 'float_aa':float_aa, 
             'bool_a':bool_a, 'bool_a':bool_a, 'bool_c': bool_c, 
             'str_a':str_a, 'str_b':str_b, 'str_c':str_c}


Now let's define a function to verify that both are the same type. We will assume that the first variable (`x`) is known to be the desired type and value and (`y`) is the variable we're checking against it. Here's the plan:
- Check if `y` is an instance of whatever class `x` is an instance of.
 - If there is a mismatch, return `False`.
- Check to see if `x` is a `bool`, `str`, or `int` type. 
  - If it is one of those types, return `False` if the values are not the same.
- Check to see if `x` is a `float`.
  - Since `float` variables are approximations, we should specify some tolerance. If the absolute difference between `x` and `y` is less than or equal to the tolerance we will consider the values equal.
  - If the absolute difference between `x` and `y` is above the tolerance we will return `False`.
- If none of the previous steps returned `False`, return `True`

We are going to need additional variables `tol` to specify the tolerance for floats and `verbose` to toggle some debugging messages.

In [ ]:
def compare_basic(x: any, y: any, verbose: bool=False, tol: float=0.0)->bool:

  if not isinstance(y, type(x)):
    if verbose: print(f'{type(x)} x is not the same type as {type(y)} y.')
    return False

  if isinstance(x, (int, bool, str)):
    if x != y:
      if verbose: print(f'{x} != {y}')
      return 
      
  if isinstance(x, float):
    if abs(x-y) > tol:
      if verbose: print(f'{x} != {y}')
      return False

  if verbose: print(f'{x} == {y}')
  return True

Now let's go through the combinations and compare.

In [ ]:
from itertools import combinations
for k1, k2 in combinations(test_vars, 2):
  print()
  print(f'Comparing {k1} and {k2}')
  compare_basic(test_vars[k1], test_vars[k2], verbose=True)


Comparing int_a and int_b
2004 != 2012

Comparing int_a and int_c
2004 == 2004

Comparing int_a and float_a
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_a and float_b
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_a and float_aa
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_a and bool_a
2004 != True

Comparing int_a and bool_c
2004 != False

Comparing int_a and str_a
<class 'int'> x is not the same type as <class 'str'> y.

Comparing int_a and str_b
<class 'int'> x is not the same type as <class 'str'> y.

Comparing int_a and str_c
<class 'int'> x is not the same type as <class 'str'> y.

Comparing int_b and int_c
2012 != 2004

Comparing int_b and float_a
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_b and float_b
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_b and float_aa
<class 'int'> x is not the same type as <class 'float'> y.

Comparing int_

That's a lot to go through, but with the verbose output it is clear that we are getting what we expect. We should probably see if changing the tolerance has an effect on comparing `float_a` and `float_aa`

In [ ]:
compare_basic(float_a, float_aa, verbose=True, tol=0.00001)

2004.0 == 2004.000000000001


True

Changing the threshold had the desired effect of ignoring the small difference between floating point numbers.

# Comparing Collections of Objects

Python has 4 basic collection types:
- `set` - a mutable, unordered collection of immutable, hashable objects.
- `tuple` - an unmutable, ordered collection of objects.
- `list` - a mutable, ordered collection of objects.
- `dict` - an unordered collection of unmutable keys mapped to object values.

The challenge in comparing `tuple`, `list`, or `dict` collection objects is that the child objects can be collections themselves. Sets are a little different because everything in a `set` object must be hashable - i.e. the child objects must be recursively immutable. For two collections to be equal, they must meet the criteria below.

- Both collections must be of the same type.
- Both collections must contain the same number of child objects.
- `set` objects must contain the same child objects - i.e. the difference between the two sets must be the empty set.
- `list` and `tuple` objects must have equal child objects (of any type) at the same positions.
- `dict` objects must have the same keys and have equal child objects (of any type) mapped to those keys.



We will write a function to evaluate whether two collection objects `x` and `y` are equal. Due to the potential for "nesting" we must use recursively check the child objects of `list`, `tuple`, and `dict` objects for equality using the same strategy used to check the parent function.

**On Recursion**

For the purposes of this discussion, recursion is the programming design pattern where a function calls itself under certain conditions. To work properly, there must also be a "base case" where all recursive calls must eventually reach.

Consider the familiar concept of a factorial: $n! = n(n-1)!;\ 0! = 1$ Below is that same recursive mathematical function in Python.

In [ ]:
def factorial(n: int):
  if n == 0:                            # base case
    return 1
  fac =  factorial(n-1)*n               # recursive call
  print(f'{n}! = {n}({n-1})! = {fac}')  # print recursive step for demonstration purposes
  return fac                            # return statement
factorial(8)

1! = 1(0)! = 1
2! = 2(1)! = 2
3! = 3(2)! = 6
4! = 4(3)! = 24
5! = 5(4)! = 120
6! = 6(5)! = 720
7! = 7(6)! = 5040
8! = 8(7)! = 40320


40320

Now that we understand recursion, we can apply it to our use case. Here's the plan for our function to compare Python collection objects.

- Verify objects are the same type.
- If the objects are basic types (`int`, `bool`, `str`, or `float`) use the `compare_basic` function from above to compare.
  - This is one base case.
- If the objects are `set` types verify that the sets are the same size and that the set difference between them is the empty set.
- If the objects are `list` or `tuple` types verify that they are the same length and iterate through them. Recursively verify that each pair of child objects is equal.
- If the objects are `dict` types verify that they have the same keys and iterate through the objects mapped to each key in both dictionaries. Recursively verify that each pair of child objects is equal.

As before, we will need to specify a tolerance for comparison of floats and a `verbose` flag to indicate whether to print the messages or suppress them.

In [ ]:
def compare_nested(x: any, y: any, verbose: bool=False, tol: float=0.0, lvl=0)->bool:
  if not isinstance(y, type(x)):
    if verbose: print(f'{type(x)} x is not the same type as {type(y)} y. lvl={lvl}.')
    return False
  if isinstance(x, (int, float, bool, str)):
    return compare_basic(x, y, verbose=verbose, tol=tol)
  if isinstance(x, set):
    if len(x) != len(y):
      if verbose: print(f'x ({len(x)}) and y ({len(y)}) have different sizes. lvl={lvl}.')
      return False
    if len(x - y) > 0:
      if verbose: print(f'The two sets contain different items. lvl={lvl}.')
      return False
  if isinstance(x, (list, tuple)):
    if len(x) != len(y):
      if verbose: print(f'Lists x ({len(x)}) and y ({len(y)}) have different lengths. lvl={lvl}.')
      return False
    for x_i, y_i in zip(x, y):
      if not compare_nested(x_i, y_i, verbose=verbose, tol=tol, lvl=lvl+1):
        if verbose: print(f'Child objects in lists not equal. lvl={lvl}.')
        return False
  if isinstance(x, dict):
    if x.keys() != y.keys():
      if verbose: print(f'Dict keys do not match. lvl={lvl}.')
      return False
    for k in x:
      if not compare_nested(x[k], y[k], verbose=verbose, tol=tol, lvl=lvl+1):
        if verbose: print(f'Child objects in dicts not equal. lvl={lvl}.')
        return False
  if verbose: print(f'Nested Structures Equivalent. lvl={lvl}.')
  return True

Now we will define some test variables to demonstrate the function. The comments indicate how each `dict` compares to `d1`.

In [ ]:
d1 = {'k1': (2, 3), 'k2':(4, 8), 'k3': 'something', 'k4': [(2,3), (9, 12, 4)], 'k5': {'foo', 'bar', 'miz'}}
d2 = {'k1': (2, 3), 'k2':(4, 8), 'k3': 'something', 'k4': [(2,3), (9, 12, 4)], 'k5': {'foo', 'bar', 'miz'}} # same as d1
d3 = {'k1': (2, 3), 'k2':(4, 8), 'k3': 'something', 'kk4': [(2,3), (9, 12, 4)], 'k5': {'foo', 'bar', 'miz'}} # different keys
d4 = {'k1': (2, 3), 'k2':(4, 8), 'k3': 'something', 'k4': [(2,3), (9, 12, 0)], 'k5': {'foo', 'bar', 'miz'}} # different list items
d5 = {'k1': (2, 3), 'k2':(4, 8), 'k3': 'something', 'k4': [(2,3), (9, 12, 4)], 'k5': {'foo', 'bar', 'moz'}} # different sets

Now we will try it out.

In [ ]:
compare_nested(d1, d2, verbose=True)

2 == 2
3 == 3
Nested Structures Equivalent. lvl=1.
4 == 4
8 == 8
Nested Structures Equivalent. lvl=1.
something == something
2 == 2
3 == 3
Nested Structures Equivalent. lvl=2.
9 == 9
12 == 12
4 == 4
Nested Structures Equivalent. lvl=2.
Nested Structures Equivalent. lvl=1.
Nested Structures Equivalent. lvl=1.
Nested Structures Equivalent. lvl=0.


True

In [ ]:
compare_nested(d1, d3, verbose=True)

Dict keys do not match. lvl=0.


False

In [ ]:
compare_nested(d1, d4, verbose=True)

2 == 2
3 == 3
Nested Structures Equivalent. lvl=1.
4 == 4
8 == 8
Nested Structures Equivalent. lvl=1.
something == something
2 == 2
3 == 3
Nested Structures Equivalent. lvl=2.
9 == 9
12 == 12
4 != 0
Child objects in lists not equal. lvl=2.
Child objects in lists not equal. lvl=1.
Child objects in dicts not equal. lvl=0.


False

In [ ]:
compare_nested(d1, d5, verbose=True)

2 == 2
3 == 3
Nested Structures Equivalent. lvl=1.
4 == 4
8 == 8
Nested Structures Equivalent. lvl=1.
something == something
2 == 2
3 == 3
Nested Structures Equivalent. lvl=2.
9 == 9
12 == 12
4 == 4
Nested Structures Equivalent. lvl=2.
Nested Structures Equivalent. lvl=1.
The two sets contain different items. lvl=1.
Child objects in dicts not equal. lvl=0.


False

# Wrap up

That completes the introduction to comparing base Python objects. You can use these comparison functions in your debugging. You may want to modify the behavior of the `verbose` flag to only display messages on mismatches when comparing large objects.